In [21]:
import sys
import xlrd
import pandas as pd
import numpy as np
import os
import fnmatch 

INPUTS = '/Users/jiz225/Desktop/ZidLab/polarity/input' 
OUTPUTS = '/Users/jiz225/Desktop/ZidLab/polarity/outputcsv'
def makecsv(inputpath, outputpath):
    inputf = open(inputpath,'r')
    outputf = open(outputpath, 'w')
    outputf.write('Name'+ ',' + 'Orient' + ',' + 'Start' + ',' + 'End' + ',' + 'Length' + ',' + 'RelativePosition' + ',' + 'Counts' + '\n')
    while True:
        line = inputf.readline()
        if not line:
            break
        string = line.rstrip() 
        cells = string.split('\t')
        name = cells[0][:6] 
        orient = cells[0][6:7]
        start = cells[1]
        end = cells[2]
        posit = abs(int(cells[3]) - int(start))
        outputf.write(str(name) + ',' + str(orient) + ',' + str(start) + ',' + str(end) + ',' + str(length) + ',' + str(posit) + ',' + str(riboCount) + '\n')
    inputf.close()
    
inputlist = []
outputlist = []
inputfolder = os.listdir(INPUTS)
extension = "*.xls"
for doc in inputfolder:
    if fnmatch.fnmatch(doc, extension):
        #change this input and ouput file path
        inputpath = str("/Users/jiz225/Desktop/ZidLab/polarity/input/") + doc
        namelist = doc.split('.')
        name = namelist[0]
        outputpath = str("/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/") + name + str(".csv")
        inputlist.append(inputpath)
        outputlist.append(outputpath)
Gene_Polarity_all = []

In [22]:
import matplotlib.pyplot as plt
%matplotlib inline
def calculatePolarity(i, l, densityati, density):
    normDist = (2*i - (l + 1))/(l - 1)
    polarity = densityati*normDist/density
    return polarity

def PolarityPerGene(inputpath):
    df1 = pd.read_csv(inputpath)
    names = inputpath.split('.')
    #this is the final output file
    nameofcsv = names[0] + str("_polarity&gene.csv")
    Namelist = df1['Name'].tolist()
    Countslist = df1['Counts'].tolist()
    Orientlist = df1['Orient'].tolist()

    startp = 0
    for i in range (len(Namelist) - 1):
        if Namelist[i + 1] == Namelist[i]:
            Countslist[i + 1] += Countslist[i]
        else:
            for j in range (startp, i+1):
                Countslist[j] = Countslist[i]
            startp = i + 1
 
    df = pd.DataFrame({'col':Countslist})
    df1['TotalCounts'] = Countslist
    Totalcountlist = df1['TotalCounts'].tolist()
    Countsperilist = df1['Counts'].tolist()
    Lengthlist = df1['Length'].tolist()
    Positlist = df1['RelativePosition'].tolist()
    Polaritylist = []
    Genelist = []
    for i in range (len(Namelist)):
        polarityofi = calculatePolarity(Positlist[i], Lengthlist[i], Countsperilist[i], Totalcountlist[i])
        Polaritylist.append(polarityofi)
        newname = str(Namelist[i]) + str(Orientlist[i])
        Genelist.append(newname)
    df = pd.DataFrame({'col':Polaritylist})
    df1['Polarity'] = Polaritylist

    #calculate total polarity
    Gene_Polarity = {}
    geneP = df1['Polarity'].tolist()
    for i in range (len(Namelist) - 1):
        if Namelist[i + 1] == Namelist[i]:
            geneP[i + 1] += geneP[i]
            geneP[i] = -1000
        else:
            Gene_Polarity[Namelist[i]] = geneP[i]
    df = pd.DataFrame({'col':geneP})
    df1['PolarityPerGene'] = geneP
    d = {'Name':Genelist, 'Totalcount': Totalcountlist, 'PolarityPerGene': geneP}
    dfcsv = pd.DataFrame(data=d)
    dfcsv = dfcsv.replace(-1000, np.nan)
    dfcsv = dfcsv.dropna(how='any',axis=0)
    
    #df1.to_csv(nameofcsv)
    dfcsv.to_csv(nameofcsv)
    return Gene_Polarity
    

In [23]:
for i in range(len(inputlist) - 1, len(inputlist)):
    makecsv(inputlist[i], outputlist[i])
    Gene_Polarity = PolarityPerGene(outputlist[i])
    Gene_Polarity_all.append(Gene_Polarity)
print('\n Number of possible input files: ' + str(len(inputlist)))
print('\n Number of possible output csv files: ' + str(len(outputlist)))


 Number of possible input files: 4

 Number of possible output csv files: 4
